In [1]:
import yfinance as yf
import requests
import time
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
from datetime import datetime

import numpy as np

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from transformers import BertTokenizer
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datetime import datetime

from transformers import (
    BertTokenizer, 
    BertForSequenceClassification, 
    
)
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
import os

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

/Users/jiunyih/venv/p311_dsa4265/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the search parameters for cryptocurrency-related filings
url = "https://www.sec.gov/cgi-bin/browse-edgar"
params = {
    "company": "Grayscale Bitcoin Cash Trust",  # Searching for Grayscale Bitcoin Cash Trust
    "match": "starts-with",
    "filenum": "",
    "State": "",
    "Country": "",
    "SIC": "",
    "myowner": "exclude",
    "action": "getcompany",
    "type": "10-K", # 10-Q for quarter report    
}

# Add a User-Agent header to comply with SEC's guidelines
headers = {
    "User-Agent": "YourAppName/1.0 (your-email@example.com)"
}

# Send the request to the SEC EDGAR API
response = requests.get(url, headers=headers, params=params)

# Parse the HTML response
soup = BeautifulSoup(response.text, "html.parser")

# Extract the links or filings
filing_links = []
for link in soup.find_all("a", href=True):
    # Only add links that end with "-index.htm"
    if link['href'].endswith("-index.htm"):
        filing_links.append(link['href'])

all_filling_links = []
# Print the filtered links
for filing in filing_links:
    all_filling_links.append(f"https://www.sec.gov{filing}")
    print(f"Filing link: https://www.sec.gov{filing}")

# Add a delay between requests to avoid rate limit issues
time.sleep(1)

Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017024104235/0000950170-24-104235-index.htm
Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017023045923/0000950170-23-045923-index.htm
Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017022018010/0000950170-22-018010-index.htm


In [3]:
docs_dic = {}

for link in all_filling_links:
    try:
        # Generate document link
        doc = link.replace("-index", "").replace("htm", "txt")
        print(f"🔗 Generated Document Link: {doc}")

        # Send the GET request
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Check for request errors

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the "Period of Report"
        period_of_report = None
        for div in soup.find_all('div', class_='formGrouping'):
            if "Period of Report" in div.text:
                period_of_report = div.find('div', class_='info').text.strip()

        # Format the period_of_report
        formatted_period_of_report = period_of_report.replace('-', '') if period_of_report else "N/A"
        print(f"✅ Formatted Period of Report: {formatted_period_of_report}")

        # Extract necessary parts from the link
        parts = link.split('/')
        cik = parts[6]  
        filing_identifier = parts[7]  

        # Construct the new URL
        new_link = f"https://www.sec.gov/ix?doc=/Archives/edgar/data/{cik}/{filing_identifier}/bchg-{formatted_period_of_report}.htm"
        print(f"🔗 Generated Report Link: {new_link}\n")

        # Store in the dictionary
        docs_dic[period_of_report] = {
            "period_of_report": formatted_period_of_report,
            "report_link": new_link,
            "doc_link": doc
        }

    except Exception as e:
        print(f"⚠️ Error processing {link}: {e}")

# Print the final dictionary
print(docs_dic)

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017024104235/0000950170-24-104235.txt
✅ Formatted Period of Report: 20240630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017024104235/bchg-20240630.htm

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017023045923/0000950170-23-045923.txt
✅ Formatted Period of Report: 20230630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017023045923/bchg-20230630.htm

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017022018010/0000950170-22-018010.txt
✅ Formatted Period of Report: 20220630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017022018010/bchg-20220630.htm

{'2024-06-30': {'period_of_report': '20240630', 'report_link': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017024104235/bchg-20240630.htm', 'doc_link'

In [ ]:
from sec_edgar_downloader import Downloader

# Provide your email address (as required by SEC)
dl = Downloader(email_address="your_email@example.com", download_folder="./sec_data", company_name="school")

dl.get(
    "10-K",
    ticker_or_cik = "AAPL",
    after = "2015-01-01"
)

dl.get(
    "10-K",
    ticker_or_cik = "MSFT",
    after = "2015-01-01"
)


10

In [20]:
with open("sec_data/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/full-submission.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

raw_text[:1000]

'<SEC-DOCUMENT>0000320193-17-000070.txt : 20171103\n<SEC-HEADER>0000320193-17-000070.hdr.sgml : 20171103\n<ACCEPTANCE-DATETIME>20171103080137\nACCESSION NUMBER:\t\t0000320193-17-000070\nCONFORMED SUBMISSION TYPE:\t10-K\nPUBLIC DOCUMENT COUNT:\t\t97\nCONFORMED PERIOD OF REPORT:\t20170930\nFILED AS OF DATE:\t\t20171103\nDATE AS OF CHANGE:\t\t20171103\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAPPLE INC\n\t\tCENTRAL INDEX KEY:\t\t\t0000320193\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tELECTRONIC COMPUTERS [3571]\n\t\tIRS NUMBER:\t\t\t\t942404110\n\t\tSTATE OF INCORPORATION:\t\t\tCA\n\t\tFISCAL YEAR END:\t\t\t0930\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-36743\n\t\tFILM NUMBER:\t\t171174673\n\n\tBUSINESS ADDRESS:\t\n\t\tSTREET 1:\t\tONE INFINITE LOOP\n\t\tCITY:\t\t\tCUPERTINO\n\t\tSTATE:\t\t\tCA\n\t\tZIP:\t\t\t95014\n\t\tBUSINESS PHONE:\t\t(408) 996-1010\n\n\tMAIL ADDRESS:\t\n\t\tSTREET 1:\t\tONE INFINITE LOOP\n\t

In [23]:
import re

# Remove line breaks and extra spaces
cleaned_text = re.sub(r'\s+', ' ', raw_text)

# Optional: Try to find section headers like "Item 1A. Risk Factors"
risk_section = re.search(r'Item\s+1A\.*\s+Risk Factors(.*?)(Item\s+1B\.|\Z)', cleaned_text, re.IGNORECASE | re.DOTALL)
if risk_section:
    risk_text = risk_section.group(1)

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Split into overlapping chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.create_documents([raw_text])

In [28]:
docs

[Document(metadata={}, page_content='<SEC-DOCUMENT>0000320193-17-000070.txt : 20171103\n<SEC-HEADER>0000320193-17-000070.hdr.sgml : 20171103\n<ACCEPTANCE-DATETIME>20171103080137\nACCESSION NUMBER:\t\t0000320193-17-000070\nCONFORMED SUBMISSION TYPE:\t10-K\nPUBLIC DOCUMENT COUNT:\t\t97\nCONFORMED PERIOD OF REPORT:\t20170930\nFILED AS OF DATE:\t\t20171103\nDATE AS OF CHANGE:\t\t20171103\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAPPLE INC\n\t\tCENTRAL INDEX KEY:\t\t\t0000320193\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tELECTRONIC COMPUTERS [3571]\n\t\tIRS NUMBER:\t\t\t\t942404110\n\t\tSTATE OF INCORPORATION:\t\t\tCA\n\t\tFISCAL YEAR END:\t\t\t0930\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-36743\n\t\tFILM NUMBER:\t\t171174673\n\n\tBUSINESS ADDRESS:\t\n\t\tSTREET 1:\t\tONE INFINITE LOOP\n\t\tCITY:\t\t\tCUPERTINO\n\t\tSTATE:\t\t\tCA\n\t\tZIP:\t\t\t95014\n\t\tBUSINESS PHONE:\t\t(408) 996-1010\n\n\tMAIL ADDRESS:\t\n\t